# Prediction - Supervised Learning

## Instances and Labels
Instances are presented by their attributes
$$
{\bf x}=(x_{1},..., x_{k})\in{\mathcal{X}}, \mathcal{X} = \mathcal{X}\times\cdot\cdot\times\mathcal{X}
$$

An instance can have a class or a value. An instance whose class/value is known is called **labeled**. Labels in our math notation are portrayed by the $\mathcal{L}$ set.
$$
(x,y) \in{\mathcal{X}\times\mathcal{L}}
$$

Assume that labels are assigned according to some unknown pattern called a labeling function, which is a mapping from an instance to a label.

$$
l:\mathcal{X} \rightarrow \mathcal{L}, l(x) = y
$$

- if $\mathcal{L} \subset \mathbb{Z}^1$ then $l$ is a classification function.
- if $\mathcal{L} \subset \mathbb{R}$ then $l$ is a regression function.

The problem is that this labeling function is unknown. But even though it is not known, we have observed a sample of instances with their labels. This set of instances is called a training sample.

$$
\mathcal{S}^{tr} = \{(x,y)|x \in\mathcal{X}, y \in\mathcal{L}\}
$$

The solution is to model a function $m: \mathcal{X} \rightarrow \mathcal{L}, m(x) = \hat{y}$ which is as close to $l$ as possible.

## Approximating $l$

In the slides, the example of a linear regression model is taken. 

In this case, $m$ is defined by its type (by type we refer to the x parameter of the function) and parameters $\Theta$.

$$
    m^\Theta(x = (x_1,...,x_k)) = \Theta_0 + \Theta_1x_1 + ... + \Theta_kx_k
$$

Where k is the number of attributes the instance x has, and $\Theta$ is defined as:

$$
    \Theta = (\Theta_0, \Theta_1, ..., \Theta_k)
$$

In order to measure if $m$ approximates $l$ well, the example of an empirical error is given.

There is also a regression loss function called $l_{r}$, but this is just the squared loss.

$$
l_r(y, m^\Theta(x)) = (y - m^\Theta(x))^2
$$

$$
e r r(m^{\Theta},{\cal S}^{t r})=\sum_{({\bf x},y)\in{\cal S}^{t r}}l_{r}(y,m^{\Theta}({\bf x}))=\sum_{({\bf x},y)\in{\cal S}^{t r}}(y-m^{\Theta}({\bf x}))^{2}
$$

This error function is the sum if the regression losses $l_{r}(y,m^{\Theta}({\bf x}))$ for each pair $(x,y)$

Let's take a concrete example (basic linear regression, with $\Theta_0$ representing the bias/y-intercept, and $\Theta_1$ representing the direction/slope):

$$
    m^\Theta(x = (x_1)) = \Theta_0 + \Theta_1x_1
$$

$m$ approximates $l$ with an error, in the slides the error was stochastic, meaning that the error is random and in our case, is part of a random distribution centered at 0 with a standard deviation of $\sigma^2$.

$$
    y = l(x) = m^\Theta(x) + \epsilon, \epsilon \sim \mathcal{N}(0, \sigma^2)
$$

thus the probability of getting the correct approximations $y$ given ${\bf x}$ is in the normal distribution centered at $\Theta_1 + \Theta_1x_1$ with std. dev. $\sigma^2$

$$
p(y|{\bf x}) = \mathcal{N}(\Theta_0 + \Theta_1x_1 , \sigma^2)
$$

The likelyhood (probability of getting $S^{tr}$ given $\Theta$) is:

$$
L_{S^{t r}}(\Theta)=\prod_{({\bf x},y)\in{\cal S}^{t r}}p({\bf x},y|\Theta)=\prod_{({\bf x},y)\in{\cal S}^{t r}}p(y|{\bf x},\Theta)p({\bf x}|\Theta)
$$

The likelihood is decomposed into two parts:
- $p(y|{\bf x},\Theta)$: The probability of observing $y$ given $x$ and the parameters $\Theta$. This is the probability distribution of $y$ given $x$ under the assumed model.
- $p(\bf{x}|\Theta)$: The probability of observing $x$ given the parameters $\Theta$. This part reflects any prior information or assumptions about the distribution of the input data.

Modeling means to choose a type of m and to find its parameters $\Theta$
such that $L_{S^{t r}}(\Theta)$ is maximal.

$$
\prod_{({\bf x},y)\in{\cal S}^{t r}}p({\bf x},y)=\prod_{({\bf x},y)\in{\cal S}^{t r}}p(y|{\bf x})p({\bf x})=\prod_{({\bf x},y)\in{\cal S}^{t r}}p(y|{\bf x})\prod_{({\bf x},y)\in{\cal S}^{t r}}p({\bf x})
$$

Since $p(x)$ does not depend on $\Theta$, it's enough to minimize the conditional likelihood.

$$
L_{S t r}^{c o n d}(\Theta)=\prod_{({\bf x},y)\in{\cal S}^{t r}}p(y|{\bf x})=\prod_{({\bf x},y)\in{\cal S}^{t r}}\frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(y-m^{\Theta}({\bf x}))^{2}}{2\sigma^{2}}}
$$

Maximizing the formula above is the same as maximizing the conditional log-likelihood. This is true because the log function is monotonic, and it does not change the location of the maximum of the function. It is also more computationally efficient for computers to use the log-likelihood, as it simplifies all the multiplications to additions, which prevents underflow.

$$
l n L_{S^{t r}}^{c o n d}(\Theta)=\sum_{({\bf x},y)\in{\cal S}^{t r}}l n\Bigl({\frac{1}{\sqrt{2\pi}\sigma}}e^{-{\frac{(y-m^{\Theta}({\bf x}))^{2}}{2\sigma^{2}}}}\Bigr)\propto\operatorname*{min}_{({\bf x},y)\in{\cal S}^{t r}}(y-m^{\Theta}({\bf x}))^{2}
$$

The logarithm of the conditional likelihood is proportional to this minimum squared error term. This proportionality suggests that, under certain assumptions and conditions (such as the Gaussian distribution assumption in the likelihood), maximizing the log-likelihood is related to minimizing the squared error. This connection is often used in the context of maximum likelihood estimation and linear regression, where minimizing the squared error is equivalent to maximizing the likelihood of the observed data given the model.

## Gradient descent optimization

for more variables closed form solutions are bothersome
- How to find a minimum of an “objective” (i think he means cost/loss function by this) function $f(\Theta)$?
- assume $f$ is differentiable and convex

In [1]:
from typing import Callable, Iterable, Optional
from itertools import repeat
import numpy as np 
from numpy.typing import NDArray

class StoppingCriteria:
    """
    Gradient descent stopping criteria class
    """
    minimum_delta: Optional[float] # epsilon in the slides, mininum rate of change between old and new weights to continue iteration 
    n_iter: Optional[int] # Maximum number of iterations

    def __init__(self, minimum_delta, n_iter) -> None:
        if not minimum_delta and not n_iter:
            raise ValueError("One or both of minimum_delta or num_iter must be supplied as parameters")
        self.minimum_delta = minimum_delta
        self.n_iter = n_iter        

def gradient_descent(
        gradient: Callable[[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]], NDArray]
      , x: NDArray[np.float64]
      , y: NDArray[np.float64]
      , start: NDArray[np.float64]
      , learn_rate: float
      , stop_criteria: StoppingCriteria
      ):
    """
    1. gradient: is the function or any Python callable object which accepts the inputs
       , the outputs and the weights of a functions and returns the gradient of the function you're trying to minimize.
    2. x: The inputs of the linear regression problem
    3. y: The outputs corresponding to the elements of the x array
    4. start: is the point where the algorithm starts its search, given as an NDArray. This is the Theta in the 
              slides that he said to initialize not with zeros
    5. learn_rate: is the learning rate that controls the magnitude of the vector update.
    6. stop_criteria: Small object in which the stop criteria are passed. The 2 stop criteria are:
                           1. Maximum number of iterations
                           2. Minimum amount of change between iterations allowed
                      If one of these thresholds is violated, the optimization stops. 
    """
    n_iter = stop_criteria.n_iter
    minimum_delta = stop_criteria.minimum_delta

    if x.shape[0] != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")

    vector = start
    for _ in range(n_iter) if n_iter else repeat(1): # repeat(1) is used to create an infinite iterator, the value passed to repeat is arbitrary
        diff = -learn_rate * np.array(gradient(x, y, vector), np.float64)
        if np.all(np.abs(diff) <= minimum_delta): # if change is smaller than the minimum allowed change, stop optimizing
            break
        vector += diff
    return vector if vector.shape else vector.item()

As an example, I'll try minimizing the Sum of Squared Residuals function for an X and Y value I'll choose below, and some starting weights which are picked randomly from 0-1.

SSR for a line approximation function $f$ looks like this:

$$
    SSR(x, y, \Theta) = \sum_i(y_i - f(x_i, \Theta))^2
$$

And its gradient (we will partially derivate with respect to $\Theta_0$ and $\Theta_1$) looks like this: (Sadly the gradient needs to be analytically derived, and im not writing all the steps here)

$$
    \bigtriangledown{}SSR = \begin{bmatrix} 
                                mean(\Theta_0 + \Theta_1x_i - y_i)\\
                                mean((\Theta_0 + \Theta_1x_i - y_i)x_i)
                            \end{bmatrix}
$$

In [3]:
def ssr_gradient(x: NDArray, y: NDArray, b: NDArray):
    res = b[0] + b[1] * x - y
    return np.array([res.mean(), (res * x).mean()])

x = np.array([5, 15, 25, 35, 45, 55])
y = np.array([5, 20, 14, 32, 22, 38])

gradient_descent(ssr_gradient, x, y, start=np.array([0.5, 0.5], dtype=np.float64)
                 , learn_rate=0.0008, stop_criteria=StoppingCriteria(1e-06, 100_000))

array([5.62822349, 0.54012867])

So for the points described in x and y, the best weights for our hyperparameters were the ones displayed in the output of the cell above.

## Stochastic gradient descent

Stochastic gradient descent algorithms are a modification of gradient descent. In stochastic gradient descent, you calculate the gradient using just a random small part of the observations instead of all of them. In some cases, this approach can reduce computation time.

Batch stochastic gradient descent is somewhere between ordinary gradient descent and the online method. The gradients are calculated and the decision variables are updated iteratively with subsets of all observations, called minibatches. This variant is very popular for training neural networks.

If you were to write an sgd() function, it would be very similar to gradient_descent() but it would use randomly selected minibatches to move along the search space.

## Prediction

The aim is not to describe the data but rather to predict labels on yet
unseen instances.

Regression generalization error:

$$
e r r(m^{\Theta})=E_{({\bf x},y)}\{l_{r}(y,m^{\Theta}({\bf x}))\}=\int\limits_\mathcal{X}\int\limits_\mathcal{L} l_{r}(y,m^{\Theta}({\bf x}))p({\bf x},y)\,\mathrm{d}y\mathrm{d}{\bf x}
$$

In simpler terms, the generalization error is calculated by taking the average (expected value) of the loss function over all possible pairs of input-output $({\bf x},y)$ according to the underlying probability distribution $p({\bf x},y)$.

Classification generalization error:

$$
e r r(m^{\Theta})=E_{({\bf x},y)}\{l_{r}(y,m^{\Theta}({\bf x}))\}=\int\limits_\mathcal{X} \sum_{c \in \mathcal{L}} l_{c}(y,m^{\Theta}({\bf x}))p({\bf x},y = c)\,\mathrm{d}y\mathrm{d}{\bf x}
$$

In simpler terms, this classification error formula is calculating the average (expected value) of the classification loss over all possible pairs of input and true labels, with the additional consideration of multiple classes (summing over $c$). It quantifies how well the model $m^Θ$ classifies input instances into different classes.

This type of error calculation is common in classification problems where the output $y$ represents a class label, and $lc$​ is a loss function specific to classification, such as cross-entropy loss.

Bayes predictor minimizes the generalization error:
$$
    m_B = \arg \min err(m^\Theta)
$$

## Regularization

The aim is to achieve low generalization error of the model
- it means, describe the available data (training data) as well as possible
- but also, don’t fit the model to the noise in the data
- i.e. try to get a smooth model

Regularized linear regression objective function:

$$
f(\Theta) = \sum_{({\bf x},y) \in \mathcal{S}^{tr}} (t - m^\Theta(x))^2 + \lambda ||\Theta||^2
$$

Where $\sum_{({\bf x},y) \in \mathcal{S}^{tr}} (t - m^\Theta(x))^2$ is the empirical error and  $\lambda ||\Theta||^2$ is the regularization term.

Quiz question: Write down the objective function for regularized linear regression and indicate what are its hyperparameters.

## Model evaluation

According to $\Theta$, we can have many different models $m^\Theta$, and each model can be trained with many different training samples. But which ones to choose, and how do we choose them?

There are 2 main metrics:
- Bias: Measures how $m^{\Theta,\mathcal{S^{tr_2}}}, m^{\Theta,\mathcal{S^{tr_2}}}, ... ,m^{\Theta,\mathcal{S^{tr_m}}}$ differs from $l$ and determines how generic the model $m^\Theta$ is.
$$
    bias_{m^\Theta}^2({\bf x}) = (l({\bf x}) - {\bf E}_{\mathcal{S}^{tr}}\{m^{\Theta,\mathcal{S}^{tr}}\})^2
$$
- Variance: Measures how $m^{\Theta,\mathcal{S^{tr_2}}}, m^{\Theta,\mathcal{S^{tr_2}}}, ... ,m^{\Theta,\mathcal{S^{tr_m}}}$ differs from each other and determines how stable the model $m^\Theta$ is.
$$
v a r i a n c e_{m^\Theta}({\bf x})={\bf E}_{S^{t r}}\{~(\,m^{\Theta,S^{t r}}({\bf x})-{\bf E}_{S^{t r}}\{m^{\Theta,S^{t r}}({\bf x})\}\,)^{2}\}
$$

Underfitting: Low variance, high bias. Means the model is too general

Overfitting: Low bias, high variance. Means the model is too specific

Usually, the bias decreases with the complexity of the model, while
variance increases with the complexity of the model. Thus, we need to
find a tradeoff model, which is not too general nor too specific.

# What happens if we sum up the bias and the variance?

From now on we will denote $m^\Theta$ as $\hat{y}$

$$
bias_{m^\Theta}^2(x) + variance_{m^\Theta}(x) = {\bf E}\{(l(x) - \hat{y})^2\}
$$

We get the expected squared error of the model over all training
samples w.r.t. the labeling.

## Noise in sampling

$$
noise({\bf x}) =  {\bf E}_{({\bf x}, y)}\{(y - l(x))^2\}
$$

Usually, we assume a normally distributed sampling error $\epsilon \sim \mathcal{N}(0, 1)$, thus:
$$
{\bf E}_{({\bf x},y)}\{y\} = l(x)
$$

Now if we sum up the bias, variance and noise we get:
$$
bias_{m^\Theta}^2(x) + variance_{m^\Theta}(x) + noise({\bf x})
$$
$$
{\bf E}_{S^{tr}}\{(l(x) - \hat{y})^2\} + noise({\bf x}) = {\bf E}_{S^{tr}}\{{\bf E}_{({\bf x}, y)}\{(y - \hat{y})^2\}\}
$$

We get the expected squared error of the model over all training
samples and all instances w.r.t. the observed labeling.

## Test set, RMSE and MAE

Test set: $S^{te} \subset \mathcal{X} \times \mathcal{L} \setminus \mathcal{S}^{tr}$

Root mean square error (regression):

$$
rmse(m^{\Theta,S^{t r}}({\bf x}),S^{t e})=\sqrt{\frac{\sum_{({\bf x},y)\in S^{t e}}(m^{\Theta,S^{t r}}({\bf x})-y)^{2}}{|{\mathcal{S}}^{t e}|}}
$$

Mean absolute error (classification)

$$
mae(m^{\Theta,S^{t r}}({\bf x}),S^{t e})=\frac{\sum_{({\bf x},y)\in S^{t e}}I(m^{\Theta,S^{t r}}({\bf x}) \ne y)}{|{\mathcal{S}}^{t e}|}
$$

Where $I(\bullet) = 1$ if condition $(\bullet)$ holds, else = 0

## K-fold cross-validation

The point of K-fold cross validation is to choose the best hyper-parameters which minimize an error function for a model, by training the model based on splits of the training set.

1. Split the training sample $\mathcal{S}^{tr} = \bigcup\limits_{k} \mathcal{S}_k^{tr}$
2. choose those hyper-parameters $\Xi$ such that: ($\mathcal{S}^{tr}_i$ is called a validation fold)
$$
    \Xi = \arg \min \{\frac{1}{k}\sum\limits_{i=1}^{k}err(m^{\Theta,\Xi,\cup_{i\le{}j\le{}k,j\ne{}i} \mathcal{S}^{tr}_j})\}
$$
3. Re-learn the final $m^\Theta$ using $\Xi$ on the whole training set $S^{tr}$

## Bayes Classifier

$C_1, ..., C_k$ are mutually exclusive and exhaustive classes.

1. Prior probability $P(C_i)$
    - Probability that an arbitrary instance is labeled with class $C_i$
2. Likelihood $P(x|C_i)$
    - Probability that an arbitrary instance of class $C_i$ is associated with the instance $x$
3. Evidence $P(x)$
    - Probability that the instance $x$ is seen regardless of its class
4. Posterior probability $P(C_i|x)$
    - Probability that the instance $x$ is labeled with class $C_i$

$$
    P(C_i | x) = \frac{P(x | C_i)P(C_i)}{P(c)}
$$

for $x$ predict $C_i$ for which $P(C_i|x)$ is maximal.

## Discriminant function

in case of K classes, classification can be seen as an implementation of
K discriminant functions g1(x), . . . , gK (x) such that
- for x predict Ci for which gi(x) is maximal

## Logistic regression

The model for logistic regression is defined as:

$$
 P(C = 1 | {\bf x}) = s({\bf w}^T{\bf x} + w_0) + \epsilon = \frac{1}{1 + e^{-({\bf w}^T{\bf x} + w_0)}} + \epsilon
$$

$$
s(t) = \frac{1}{1+e^{-t}}
$$

Where the function $s(t)$ is called the sigmoid logistic function and $\bf{w}$ and $w_0$ are model parameters